### Introduction
Besides using loss or AUC to optmize the model,
this notebook provide another way to fine tune.
1. Using Pytorch Dataloader to read data.
2. Save model weight every n epochs.
3. Compute utility score with respect to each weight.  

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
import dask.dataframe as dd
from glob import glob
import os

`create_mlp` is copied from <a link='https://www.kaggle.com/tarlannazarov/own-jane-street-with-keras-nn'>
OWN Jane Street with Keras NN<a>

In [ ]:
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)
    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

### Convert csv file to Parquet File  
To save time, we just provide the code here and use ```add data``` to import results.
```python
data = pd.read_csv ('../input/jane-street-market-prediction/train.csv')
features_columns = ['feature_{}'.format(i) for i in range(1,130)]
resp_columns = ['resp_{}'.format(i) for i in range(1,5)] + ['resp']
for i in tqdm(data['date'].unique()):
    data[data['date']==i].to_parquet('./date_{}.parquet'.format(i))
```


In [ ]:
import dask.dataframe as dd

data = dd.read_parquet('../input/janestreetparquetdata/date*.parquet')

features = ['feature_{}'.format(i) for i in range(130)]
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

train = data.compute()
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
f_mean = train[features].mean().values
train = train.dropna()

X_train = train[features].values
y_train = (train[resp_cols]> 0).astype(int).values

### Tensorflow ModelCheckpoint

Add ModelCheckpoint to record weights.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

SEED = 1111
tf.random.set_seed(SEED)
np.random.seed(SEED)

hidden_units = [150, 150, 150]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

epochs = 250
batch_size = 5000
save_every_n_epochs = 10
save_freq = (len(X_train)//batch_size)*save_every_n_epochs

clf = create_mlp(
    len(features), len(resp_cols), hidden_units, dropout_rates, label_smoothing, learning_rate
    )
checkpoint_path = "./cp-{epoch:04d}.ckpt"
checkpoint = ModelCheckpoint(checkpoint_path,save_weights_only=True,save_freq=save_freq )
clf.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,callbacks=[checkpoint])
clf.save_weights('./SimpleMLP.ckpt')

### Pytorch Dataloader
The following code creates Pytorch Dataloader.  
reading the parquet files that has been preprocessed.  
If you want to change the output, just modify `__getitem__` function.

In [ ]:
from torch.utils.data import Dataset, DataLoader
class TrainData(Dataset):    
    def __init__(self,file_name,root_dir,predict=False):
        self.file_name = file_name
        self.root_dir = root_dir
        self.feature = ['feature_{}'.format(i) for i in range(130)]
        self.resp = ['resp_{}'.format(i) for i in range(1,5)]+['resp']
        self.prediction = predict
    def __len__(self):
        return len(glob(os.path.join(self.root_dir,'*.parquet')))
    def __getitem__(self, idx):
        data = pd.read_parquet(os.path.join(self.root_dir,self.file_name+'_{}.parquet'.format(idx)))
        X = data[self.feature].values
        W = data['weight'].values
        R = data['resp'].values
        return X,W,R

In [ ]:
dataset = TrainData('date','../input/janestreetparquetdata/',predict = True)
weight_path = glob('./*.ckpt.index')
weight_path = [os.path.basename(each).split('.')[0] for each in weight_path]
weight_path.sort()
for path in weight_path:
    clf.load_weights('./{}.ckpt'.format(path))
    p = []
    for i in range(len(dataset)):
        net_input,weight,resp= dataset[i]
        net_input = np.nan_to_num(net_input)+f_mean*(np.isnan(net_input).astype(int))
        net_prediction = clf.predict(net_input)
        pre = (np.median(net_prediction,axis=1)>0.5).astype(int)
        p.append((weight*resp*pre).sum())
    result = pd.DataFrame(data={'p':p})
    result ['p2'] = result['p']**2
    t = (result['p'].sum()/np.sqrt(result['p2'].sum()))*(np.sqrt(250/len(dataset)))
    print(path,min(max(t,0),6)*result['p'].sum())

In [ ]:
selection = 'cp-0200'
clf.load_weights('./{}.ckpt'.format(selection))

In [ ]:
import janestreet
env = janestreet.make_env()
th = 0.5
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        x_tt = np.nan_to_num(x_tt)+f_mean*(np.isnan(x_tt).astype(int))
        pred = np.median(clf(x_tt, training=False))
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)